# Analyzing VIIRS imagery in AWS

This notebook focuses on two nighttime lights analyses: zonal statistics and timelapse mapping

In [2]:
import sys, os, importlib, json
import rasterio
import imageio

import pandas as pd
import geopandas as gpd

from shapely.geometry import shape
from rasterio import MemoryFile
from contextlib import contextmanager

sys.path.insert(0, "/home/wb411133/Code/gostrocks/src")

import GOSTRocks.rasterMisc as rMisc
import GOSTRocks.ntlMisc as ntl
from GOSTRocks.misc import tPrint

%load_ext autoreload
%autoreload 2

/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [3]:
# Get a list of the VIIRS images in S3. This example leverages the GOST teams S3 bucket
ntl_files = ntl.aws_search_ntl()
ntl_files

['https://globalnightlight.s3.amazonaws.com/composites/j01_202306_ops/DNB_j01_20230601-20230630_global_ecm-slcorr_v10_ops.avg_rade9.tif',
 'https://globalnightlight.s3.amazonaws.com/composites/j01_202307_ops/DNB_j01_20230701-20230731_global_ecm-slcorr_v10_ops.avg_rade9.tif',
 'https://globalnightlight.s3.amazonaws.com/composites/j01_202309_ops/DNB_j01_20230901-20230930_global_ecm-slcorr_v10_ops.avg_rade9.tif',
 'https://globalnightlight.s3.amazonaws.com/composites/j01_202310_ops/DNB_j01_20231001-20231031_global_ecm-slcorr_v10_ops.avg_rade9.tif',
 'https://globalnightlight.s3.amazonaws.com/composites/j01_202311_ops/DNB_j01_20231101-20231130_global_ecm-slcorr_v10_ops.avg_rade9.tif',
 'https://globalnightlight.s3.amazonaws.com/composites/j01_202312_ops/DNB_j01_20231201-20231231_global_ecm-slcorr_v10_ops.avg_rade9.tif',
 'https://globalnightlight.s3.amazonaws.com/composites/j01_202401_ops/DNB_j01_20240101-20240131_global_ecm-slcorr_v10_ops.avg_rade9.tif',
 'https://globalnightlight.s3.amaz

## Extract and clip VIIRS data

<ipython-input-8-dcf41dee71a0>:1: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  country['geometry'] = country['geometry'].buffer(1)
/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [11]:
iso3='KHM'
out_folder = f"/home/wb411133/temp/VIIRS_{iso3}"
if not os.path.exists(out_folder):
    os.makedirs(out_folder)
# get country extent from geopandas
world_filepath = gpd.datasets.get_path('naturalearth_lowres')
world = gpd.read_file(world_filepath)
country = world[world.iso_a3 == iso3]
country['geometry'] = country['geometry'].buffer(1)

for ntl_file in ntl_files[-4:]:
    out_file = os.path.join(out_folder, os.path.basename(ntl_file))
    if not os.path.exists(out_file):
        curR = rasterio.open(ntl_file)
        rMisc.clipRaster(curR, country, outFile=out_file, crop=False)
        tPrint(out_file)
    
    

<ipython-input-11-b320a2eee7d4>:9: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  country['geometry'] = country['geometry'].buffer(1)


11:02:41	/home/wb411133/temp/VIIRS_KHM/DNB_npp_20231001-20231031_global_ecm-slcorr_v10_ops.avg_rade9.tif
11:02:42	/home/wb411133/temp/VIIRS_KHM/DNB_npp_20231101-20231130_global_ecm-slcorr_v10_ops.avg_rade9.tif
11:02:42	/home/wb411133/temp/VIIRS_KHM/DNB_npp_20231201-20231231_global_ecm-slcorr_v10_ops.avg_rade9.tif
11:02:43	/home/wb411133/temp/VIIRS_KHM/DNB_npp_20240101-20240131_global_ecm-slcorr_v10_ops.avg_rade9.tif


In [5]:
rMisc.clipRaster?

## Zonal Statistics

In [3]:
# Run zonal statistics against the define admin (in_zones)
in_zones = "/home/public/Data/GLOBAL/ADMIN/Admin0_Polys.shp"
inD = gpd.read_file(in_zones)
inD = inD.loc[inD['ISO3'] == "URY"]
inD = inD.to_crs("epsg:4326")

for cur_tif in ntl_files:
    res = rMisc.zonalStats(inD, cur_tif, minVal=0.05, reProj=True)
    res = pd.DataFrame(res,columns=['SUM','MIN','MAX','MEAN'])
    inD[cur_tif.split("/")[5]] = res['SUM']
    tPrint(os.path.basename(cur_tif))
    
pd.DataFrame(inD.drop(['geometry'], axis=1)).to_csv(f"{in_zones[:-4]}_NTL.csv")

11:30:35	DNB_npp_20120119-20120131_global_vcm-slcorr_v10_rp2.avg_rade9.tif
11:30:35	DNB_npp_20120201-20120229_global_vcm-slcorr_v10_rp2.avg_rade9.tif
11:30:36	DNB_npp_20120301-20120331_global_vcm-slcorr_v10_rp2.avg_rade9.tif
11:30:38	DNB_npp_20120401-20120430_global_vcm-slcorr_v10_rp2.avg_rade9.tif
11:30:38	DNB_npp_20120501-20120531_global_vcm-slcorr_v10_rp2.avg_rade9.tif
11:30:39	DNB_npp_20120601-20120630_global_vcm-slcorr_v10_rp2.avg_rade9.tif
11:30:40	DNB_npp_20120701-20120731_global_vcm-slcorr_v10_rp2.avg_rade9.tif
11:30:41	DNB_npp_20120801-20120831_global_vcm-slcorr_v10_rp2.avg_rade9.tif
11:30:42	DNB_npp_20120901-20120930_global_vcm-slcorr_v10_rp2.avg_rade9.tif
11:30:43	DNB_npp_20121001-20121031_global_vcm-slcorr_v10_rp2.avg_rade9.tif
11:30:44	DNB_npp_20121101-20121130_global_vcm-slcorr_v10_rp2.avg_rade9.tif
11:30:45	DNB_npp_20121201-20121231_global_vcm-slcorr_v10_rp2.avg_rade9.tif
11:30:46	DNB_npp_20130101-20130131_global_vcm-slcorr_v10_rp2.avg_rade9.tif
11:30:47	DNB_npp_20130201

11:32:28	DNB_npp_20210401-20210430_global_ecm-slcorr_v10_ops.avg_rade9.tif
11:32:29	DNB_npp_20210501-20210531_global_ecm-slcorr_v10_ops.avg_rade9.tif
11:32:30	DNB_npp_20210601-20210630_global_ecm-slcorr_v10_ops.avg_rade9.tif
11:32:31	DNB_npp_20210701-20210731_global_ecm-slcorr_v10_ops.avg_rade9.tif
11:32:32	DNB_npp_20210901-20210930_global_ecm-slcorr_v10_ops.avg_rade9.tif
11:32:33	DNB_npp_20211001-20211031_global_ecm-slcorr_v10_ops.avg_rade9.tif
11:32:34	DNB_npp_20211101-20211130_global_ecm-slcorr_v10_ops.avg_rade9.tif
11:32:35	DNB_npp_20211201-20211231_global_ecm-slcorr_v10_ops.avg_rade9.tif
11:32:36	DNB_npp_20220101-20220131_global_ecm-slcorr_v10_ops.avg_rade9.tif
11:32:37	DNB_npp_20220201-20220228_global_ecm-slcorr_v10_ops.avg_rade9.tif
11:32:38	DNB_npp_20220301-20220331_global_ecm-slcorr_v10_ops.avg_rade9.tif
11:32:39	DNB_npp_20220401-20220430_global_ecm-slcorr_v10_ops.avg_rade9.tif
11:32:40	DNB_npp_20220501-20220531_global_ecm-slcorr_v10_ops.avg_rade9.tif
11:32:41	DNB_npp_20220701

PermissionError: [Errno 13] Permission denied: '/home/public/Data/GLOBAL/ADMIN/Admin0_Polys_NTL.csv'

In [ ]:
# If you want to clip out the raster data for the below mapping, run this block

in_zones = "/home/public/Data/GLOBAL/ADMIN/Admin0_Polys.shp"
inD = gpd.read_file(in_zones)
inD = inD.loc[inD['ISO3'] == "URY"]
inD = inD.to_crs("epsg:4326")

for cur_tif in ntl_files:
    file = f'{cur_tif.split("/")[-1]}.tif'
    out_file = os.path.join(os.path.join(viirs_folder, "%s" % file))
    curR = rasterio.open(cur_tif)
    if not os.path.exists(out_file):
        rMisc.clipRaster(curR, inD, out_file)

In [ ]:
# Clip out DMSP datasets
dmsp_folder = "/home/public/Data/GLOBAL/NighttimeLights/DMSP/"
for dmsp_file in os.listdir(dmsp_folder):
    curR = rasterio.open(os.path.join(dmsp_folder, dmsp_file))
    out_file = os.path.join(os.path.join(viirs_folder, "%s" % file))
    

# Generate maps automatically

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
import numpy as np
import xarray as xr
import rioxarray as rxr
#import earthpy as et
#import earthpy.plot as ep

# Prettier plotting with seaborn
import seaborn as sns
sns.set(font_scale=1.5, style="whitegrid")

all_files = [os.path.join(viirs_folder, x) for x in os.listdir(viirs_folder)]
all_files.sort()
all_files[:5]

In [ ]:
def map_viirs(cur_file, out_file=''):
    ''' create map of viirs data
    
    INPUT
        cur_file [string] - path to input geotif
        [optional] out_file [string] - path to create output image
    '''
    # extract the year from the file name
    year = cur_file.split("_")[-1][:4]
    
    # Open the VIIRS data and reclassify 
    inR = rasterio.open(cur_file)
    inD = inR.read()
    ### TODO: play with class_bins to change the colour scale
    class_bins = [-10,0.5,1,2,3,5,10,15,20,30,40,50]
    inC = xr.apply_ufunc(np.digitize,inD,class_bins)

    # Plot the figure, remove grid and ticks
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.grid(False)
    ax.set_xticks([])
    ax.set_yticks([])
    
    ### TODO: add the year to the map, may need to experiment with the location depend on geography
    ax.text(0,5, year, fontsize=40, color='white')

    plt.margins(0,0)
    if out_file != '':
        #plt.imsave(out_file, inC[0,:,:], cmap=plt.get_cmap('magma'))
        plt.imshow(inC[0,:,:], cmap=plt.get_cmap('magma'))
        fig.savefig(out_file, dpi=100, bbox_inches='tight', pad_inches=0)
    else:
        # https://matplotlib.org/stable/tutorials/colors/colormaps.html
        plt.imshow(inC[0,:,:], cmap=plt.get_cmap('magma'))

cur_file = all_files[0]
out_file = os.path.join(out_map_folder, os.path.basename(cur_file))

print(out_file)
map_viirs(cur_file, '')
#map_viirs(cur_file, out_file)

In [ ]:
for cur_file in all_files:
    out_file = os.path.join(out_map_folder, os.path.basename(cur_file))
    map_viirs(cur_file, out_file)

In [ ]:
kwargs = {'duration':0.3}
images = []
all_tifs = [os.path.join(out_map_folder, x) for x in os.listdir(out_map_folder)]
all_tifs.sort()
for filename in all_tifs:
    images.append(imageio.imread(filename))
    
imageio.mimsave("%s_timelapse.gif" % out_map_folder, images, **kwargs)

In [ ]:
"%s_timelapse.gif" % out_map_folder